In [20]:
pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.7 MB/s eta 0:00:00


In [26]:
import openai
import os

#from dotenv import load_dotenv, find_dotenv
#_ = load_dotenv(find_dotenv()) # read local .env file

from google.colab import userdata

api_token = userdata.get('OPENAI_KEY')
from openai import OpenAI

client = OpenAI(
    api_key=api_token,  # This is the default and can be omitted
)

In [32]:
def chat(message):
  response = client.chat.completions.create(
      model = "gpt-4",
      messages=message
  )

  return response.choices[0].message.content

In [40]:
conv_log = [
    {"role":"system", "content":"You are a math tutor. Your task is to help your student learn how to solve math problems."}
]

# Part a. Solve a complex math problem

In [41]:
prompt = "Solve the following math problem step by step: What is the modulus of -4 + 3i?"
conv_log.append({"role":"user", "content":prompt})
sol = chat(conv_log)
print(sol)

The modulus of a complex number is calculated using the formula: |z| = √(a² + b²), where a is the real part of the number and b is the imaginary part of the number.

For the complex number -4 + 3i:

- The real part (a) is -4
- The imaginary part (b) is 3 

Substituting these into the formula gives:

|z| = √((-4)² + 3²)

This equals:

|z| = √(16 + 9)

Further calculation yields:

|z| = √25

The square root of 25 is 5, so: 

|z| = 5

So the modulus of -4 + 3i is 5.


In [42]:
conv_log.append({"role":"assistant", "content":sol})

# Part b. Give a PDF and website document; ask the bot to rewrite and answer questions on the given PDF and website

From my understanding, this means to give the api a pdf/website with questions on it, and ask it to rewrite said questions and answer them.

## Giving a PDF

In [43]:
import PyPDF2

In [44]:
reader = PyPDF2.PdfReader("tst.pdf")
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text()
text

'Solve x2- 2x - 2 = 0'

In [45]:
conv_log.append({"role":"user", "content":f"Rewrite and answer the question found from the following PDF content: {text}"})
pdf_sol = chat(conv_log)
print(pdf_sol)

The question is asking to solve the quadratic equation x² - 2x - 2 = 0. 

We can use the quadratic formula, x = [-b ± sqrt(b² - 4ac)] / 2a, where a, b, and c are coefficients from the quadratic equation ax² + bx + c = 0. 

For the given equation, a is 1, b is -2, and c is -2.

Let's substitute these values in the formula:

x = [2 ± sqrt((-2)² - 4*(1)*(-2))] / 2*(1)
x = [2 ± sqrt(4 + 8)] / 2
x = [2 ± sqrt(12)] / 2
x = [2 ± 2√3] / 2
x = 1 ± √3

So, the solutions to the equation x² - 2x - 2 = 0 are x = 1 + √3 and x = 1 - √3.


In [47]:
conv_log.append({"role":"assistant","content":pdf_sol})

## Giving website text

In [48]:
from bs4 import BeautifulSoup
import requests

def extract_text_from_website(url):
    # Send HTTP request to the website
    response = requests.get(url)

    # Parse the content with BeautifulSoup
    soup = BeautifulSoup(response.content, "html.parser")

    # Extract the text from the webpage
    # You can customize this to extract specific sections if needed
    paragraphs = soup.find_all('p')  # Finds all <p> tags, which usually contain the main text

    # Join the paragraphs into one string
    text = "\n".join([para.get_text() for para in paragraphs])
    return text

web_text = extract_text_from_website("https://www.princetonreview.com/college-advice/sat-practice-questions")

In [49]:
conv_log.append({"role":"user", "content":f"Given the following website content from {web_text}, rewrite and answer the first question."})
web_sol = chat(conv_log)
print(web_sol)

The first question is: 

A gas station sells regular gasoline for $2.39 per gallon and premium gasoline for $2.79 per gallon. If the gas station sold a total of 550 gallons of both types of gasoline in one day for a total of $1,344.50, how many gallons of premium gasoline were sold?

Let's solve this step by step:

Step 1: Assume the gas station sold 'x' gallons of premium gasoline.

Step 2: If 550 total gallons were sold, then the gas station must have sold 550 - x gallons of regular gasoline.

Step 3: The money earned from selling x gallons of premium gasoline is x * $2.79
The money earned from selling 550 - x gallons of regular gasoline is (550 - x) * $2.39

Step 4: Since we know the total amount of money made is $1,344.50, we can set up a equation:
x*2.79 + (550 - x)*2.39 = 1344.50

Step 5: Multiply throughout to get the equation simplified:
2.79x + 1316.5 - 2.39x = 1344.50
0.4x = 28
x = 70

Therefore, the gas station sold 70 gallons of premium gasoline.


In [50]:
conv_log.append({"role":"assistant", "content":web_sol})

# Part c. Summarization

In [51]:
conv_log.append({"role":"user", "content":"Summarize the entire conversation."})
summ = chat(conv_log)
print(summ)

The conversation includes the assistant helping the student solve various math problems. 

First, the assistant explains how to calculate the modulus of a complex number and finds the modulus of -4 + 3i to be 5.

Next, the assistant explains the quadratic formula and uses it to solve the equation x² - 2x - 2 = 0, finding two solutions: x = 1 + √3 and x = 1 - √3.

Then, the assistant is asked to rewrite and solve the first math problem from a hypothetical SAT practice test on a website. The problem involves the sale of regular and premium gasoline at a gas station, and the task is to find how many gallons of premium gasoline were sold. By setting up an equation based on provided information and solving it, the assistant concludes that the station sold 70 gallons of premium gasoline. 

Overall, the conversation is about problem-solving techniques for a variety of mathematical problems.
